In [1]:
import requests
import json
import pandas as pd

In [2]:
# Initial function that retrieves and process data from ONS
# Trade balance, imports and exports
# Final function with automatic text

def trade():
    parsedData = []
    balance_data = requests.get('https://www.ons.gov.uk/economy/nationalaccounts/balanceofpayments/timeseries/ikbj/mret/data')
    balance_json = json.loads(balance_data.content)
    imports_data = requests.get('https://www.ons.gov.uk/economy/nationalaccounts/balanceofpayments/timeseries/ikbi/mret/data')
    imports_json = json.loads(imports_data.content)
    exports_data = requests.get('https://www.ons.gov.uk/economy/nationalaccounts/balanceofpayments/timeseries/ikbh/mret/data')
    exports_json = json.loads(exports_data.content)
    tradeData = {}
    textData = {}
    data_input = 'years'
    input_int = 63

    for data_s in range(len(imports_json[data_input])):
            balance_json[data_input][data_s]['balance'] = balance_json[data_input][data_s]['value']
            balance_json[data_input][data_s]['alt-date'] = balance_json[data_input][data_s]['date']

    for data_s in range(len(imports_json[data_input])):
            balance_json[data_input][data_s]['imports'] = imports_json[data_input][data_s]['value']
            balance_json[data_input][data_s]['alt-date-s'] = imports_json[data_input][data_s]['date']

    for data_p in range(len(exports_json[data_input])):
            balance_json[data_input][data_p]['exports'] = exports_json[data_input][data_p]['value']
            balance_json[data_input][data_p]['alt-date-p'] = exports_json[data_input][data_p]['date']

    for data in balance_json[data_input][input_int:]:
            tradeData['date'] = data['date']
            tradeData['balance'] = data['balance']
            tradeData['imports'] = data['imports']
            tradeData['exports'] = data['exports']
            parsedData.append(tradeData.copy())
    last_balance = pd.to_numeric(balance_json['years'][-1]['value'])
    last_imports = pd.to_numeric(imports_json['years'][-1]['value'])
    penu_imports = pd.to_numeric(imports_json['years'][-2]['value'])
    first_imports = pd.to_numeric(imports_json['years'][63]['value'])
    last_exports = pd.to_numeric(exports_json['years'][-1]['value'])
    penu_exports = pd.to_numeric(exports_json['years'][-2]['value'])
    first_exports = pd.to_numeric(exports_json['years'][63]['value'])
    df = pd.DataFrame(data=parsedData)
    df['date'] = pd.to_datetime(df['date'])
    df['balance'] = pd.to_numeric(df['balance'])
    df['imports'] = pd.to_numeric(df['imports'])
    df['exports'] = pd.to_numeric(df['exports'])
    balance_change = round(100*(df['balance'].iloc[-1]/df['balance'].iloc[0]-1), 1)
    imports_change = round(100*(df['imports'].iloc[-1]/df['imports'].iloc[0]-1), 1)
    exports_change = round(100*(df['exports'].iloc[-1]/df['exports'].iloc[0]-1), 1)

    if (last_balance < 0):
        trade_text = 'The trade deficit was £' + str(balance_json['years'][-1]['value']) + ' million in ' + str(balance_json['years'][-1]['date']) + ", an increase of " + str(balance_change) + '% since ' + str(balance_json['years'][63]['date']) + ". "
    else:
        trade_text = 'The trade balance is positive with £' + str(balance_json['years'][-1]['value']) + ' million in ' + str(balance_json['years'][-1]['date']) + ", an increase of " + str(balance_change) + '% since ' + str(balance_json['years'][63]['date']) + ". "

    if last_imports > penu_imports:
        imports_text_1 = "Imports reached £" + str(last_imports) + " million in " + str(imports_json['years'][-1]['date']) + ". "
    else:
        imports_text_1 = "Imports decreased to £" + str(last_imports) + " million in " + str(imports_json['years'][-1]['date']) + ". "
    if last_imports > first_imports:
        imports_text_2 = "That is an increase of " + str(imports_change) + "% since " + str(imports_json['years'][63]['date']) + ". "
    imports_text_3 = imports_text_1 + imports_text_2

    if last_exports > penu_exports:
        exports_text_1 = "Exports reached £" + str(last_exports) + " million in " + str(exports_json['years'][-1]['date']) + ". "
    else:
        imports_text_1 = "Imports decreased to £" + str(last_exports) + " million in " + str(exports_json['years'][-1]['date']) + ". "
    if last_exports > first_exports:
        exports_text_2 = "That is an increase of " + str(exports_change) + "% since " + str(exports_json['years'][63]['date']) + ". "
    exports_text_3 = exports_text_1 + exports_text_2

    trade_text_final = trade_text + imports_text_3 + exports_text_3
    textData['text'] = trade_text_final
    parsedData.append(textData.copy())
    return parsedData

trade()

[{'balance': '-25171',
  'date': '2011',
  'exports': '498862',
  'imports': '524033'},
 {'balance': '-33364',
  'date': '2012',
  'exports': '501055',
  'imports': '534419'},
 {'balance': '-35367',
  'date': '2013',
  'exports': '519913',
  'imports': '555280'},
 {'balance': '-36831',
  'date': '2014',
  'exports': '518925',
  'imports': '555756'},
 {'balance': '-32370',
  'date': '2015',
  'exports': '517161',
  'imports': '549531'},
 {'balance': '-43013',
  'date': '2016',
  'exports': '547473',
  'imports': '590486'},
 {'text': 'The trade deficit was £-43013 million in 2016, an increase of 70.9% since 2011. Imports reached £590486 million in 2016. That is an increase of 12.7% since 2011. Exports reached £547473 million in 2016. That is an increase of 9.7% since 2011. '}]

In [3]:
# Initial function that retrieves and process data from ONS
# Trade Services

def services_data():
    parsedData = []
    services_data = requests.get('https://www.ons.gov.uk/economy/nationalaccounts/balanceofpayments/timeseries/ikbd/ukea/data')
    services_json = json.loads(services_data.content)
    tradeData = {}
    data_input = 'years'
    input_int = 60
    for data in services_json[data_input][input_int:]:
            tradeData['date'] = data['date']
            tradeData['value'] = data['value']
            parsedData.append(tradeData.copy())
    return parsedData

services_data()

[{'date': '2008', 'value': '49349'},
 {'date': '2009', 'value': '52961'},
 {'date': '2010', 'value': '56012'},
 {'date': '2011', 'value': '69608'},
 {'date': '2012', 'value': '75323'},
 {'date': '2013', 'value': '84416'},
 {'date': '2014', 'value': '86291'},
 {'date': '2015', 'value': '86256'},
 {'date': '2016', 'value': '92378'}]